# Card Classifier

---

#### 1. Preparing the data

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import sys

In [2]:
print('System:', sys.version)
print('PyTorch:', torch.__version__)
print('Torchvision:', torchvision.__version__)
print('Numpy:', np.__version__)
print('Pandas:', pd.__version__)

System: 3.11.13 | packaged by conda-forge | (main, Jun  4 2025, 14:52:34) [Clang 18.1.8 ]
PyTorch: 2.5.1
Torchvision: 0.20.1
Numpy: 1.26.4
Pandas: 2.3.1


In [3]:
class CardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [4]:
data_dir = '../data/train'

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [6]:
dataset = CardDataset(data_dir, transform)

In [7]:
len(dataset)

7624

In [8]:
img, lbl = dataset[1000]

In [9]:
img.shape, lbl

(torch.Size([3, 128, 128]), 6)

In [10]:
data_dir = '../data/train'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
#print(target_to_class)

In [11]:
dataloader = DataLoader(dataset, batch_size=34, shuffle=True)

In [12]:
for img, lbl in dataloader:
    break

In [13]:
img.shape

torch.Size([34, 3, 128, 128])

In [15]:
lbl

tensor([27, 17, 24, 52, 43,  7, 32,  6, 33, 36, 10, 28, 19, 42, 12,  2, 29,  5,
        27,  5, 34, 19, 52, 46,  8,  6, 16, 20, 13,  0,  9, 19,  3,  5])

---

#### 2. Creating the model

In [26]:
class CardClassifier(nn.Module):
    def __init__(self, num_classes=53):
        super(CardClassifier, self).__init__()
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        
        enet_out_size = 1280
        self.classifier = nn.Linear(enet_out_size, num_classes)

    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output

In [27]:
model = CardClassifier(num_classes=53)

In [28]:
print(str(model)[:500])

CardClassifier(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
 


In [32]:
eg_out = model(img)
eg_out.shape

torch.Size([34, 53])

---

#### 3. Training the model